In [1]:
import pickle
import pandas as pd
import os
from transformers import GPT2Tokenizer
import openai
import numpy as np
import ipdb
import re
import tqdm
from sklearn import metrics
import conlleval

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
import spacy

nlp = spacy.load("en_core_web_sm")

In [2]:
pd.set_option('max_rows',500,'max_colwidth',5000)

In [289]:
data_name = 'ddi'
seed = 33
few_shot_num = 5
number_of_test_samples = 50
selection_strategy = 'random'
sep = None
engine='davinci'
permutation = 4
verbalizer = 'natural'

if verbalizer == '':
    if permutation == 0:
        prompt_filename = '../data/{}_test_prompts_{}_{}.{}.{}'.format(data_name, selection_strategy,seed,few_shot_num,number_of_test_samples,permutation, verbalizer)
    else:
        prompt_filename = '../data/{}_test_prompts_{}_{}.{}.{}.perm_{}{}'.format(data_name, selection_strategy,seed,few_shot_num,number_of_test_samples,permutation, verbalizer)
else:
    prompt_filename = '../data/{}_test_prompts_{}_{}.{}.{}.perm_{}.{}'.format(data_name, selection_strategy,seed,few_shot_num,number_of_test_samples,permutation, verbalizer)
filename_root = prompt_filename.split('/')[-1]

In [290]:
preds_filename = '../outputs/'+filename_root+'.preds.{}'.format(engine)
df = pickle.load(open(preds_filename,'rb'))

In [24]:
preds_filename = '../data/ddi_test_prompts_random_42.5.50.perm_4.KNN.preds'
df = pickle.load(open(preds_filename,'rb'))

In [25]:
filename_root = 'ddi_test_prompts_random_42.5.50.perm_0.'

In [292]:
if (seed==42 or seed==33) and (permutation <2) and verbalizer == '':
    df[filename_root] = [e['choices'][0]['text'].strip() for e in df['gpt3_output_'+filename_root]]

In [26]:
print(df.test_ready_prompt.values[0])

Sentence: The use of antacids should be considered in place of H2 blockers or proton pump inhibitors in patients receiving SPRYCEL therapy.
Drug 1: antacids
Drug 2: H2 blockers
Interaction: none

Sentence: The use of antacids should be considered in place of H2 blockers or proton pump inhibitors in patients receiving SPRYCEL therapy.
Drug 1: antacids
Drug 2: proton pump inhibitors
Interaction: none

Sentence: The use of antacids should be considered in place of H2 blockers or proton pump inhibitors in patients receiving SPRYCEL therapy.
Drug 1: antacids
Drug 2: SPRYCEL
Interaction: none

Sentence: The use of antacids should be considered in place of H2 blockers or proton pump inhibitors in patients receiving SPRYCEL therapy.
Drug 1: H2 blockers
Drug 2: proton pump inhibitors
Interaction: none

Sentence: The use of antacids should be considered in place of H2 blockers or proton pump inhibitors in patients receiving SPRYCEL therapy.
Drug 1: H2 blockers
Drug 2: SPRYCEL
Interaction: advice

In [294]:
df.labels.unique()

array(['Recommendation', 'Effect Alteration', 'Metabolic Mechanism ',
       'No Interaction', 'Unspecified'], dtype=object)

In [12]:
new_preds = []

for pred in df[filename_root]:
    for label in df.labels.unique():
        if label.lower().startswith(pred):            
            new_preds.append(label)
            break

In [27]:
df.labels.unique()

array(['advice', 'effect', 'mechanism', 'none', 'other'], dtype=object)

In [14]:
df[filename_root] = new_preds

In [28]:
def evaluate_re_preds(df, pred_name, unique_label_dict):
    true_labels = [unique_label_dict[lab] for lab in df.labels]
    pred_labels = [unique_label_dict[lab] for lab in df[pred_name]]
    
    print(metrics.f1_score(true_labels, pred_labels, labels=list(unique_label_dict.values()), average=None))
    print(metrics.f1_score(true_labels, pred_labels, labels=list(unique_label_dict.values()), average='macro'))

In [29]:
unique_label_dict

{'advice': 0, 'mechanism': 1, 'none': 2, 'effect': 3, 'other': 4}

In [30]:
unique_label_dict = {lab : i for lab,i in zip(df.unique_labels.values[0], range(len(df.unique_labels.values[0])))}
evaluate_re_preds(df, filename_root, unique_label_dict)

[0.28571429 0.77777778 0.66666667 0.63157895 0.        ]
0.4723475355054303


In [32]:
df

,Unnamed: 0,index,sentence,label,doc,split,entity1,entity2,prompts,empty_prompts,labels,unique_labels,test_ready_prompt,ddi_test_prompts_random_42.5.50.perm_0.,gpt3_output_ddi_test_prompts_random_42.5.50.perm_0.
1065,4386,DDI-DrugBank.d48.s12.p2,The concomitant use of H2 blockers or proton pump inhibitors with SPRYCEL is not recommended.,DDI-advise,DDI-DrugBank.d48,dev,proton pump inhibitors,SPRYCEL,Sentence: The concomitant use of H2 blockers or proton pump inhibitors with SPRYCEL is not recommended.\nDrug 1: proton pump inhibitors\nDrug 2: SPRYCEL\nInteraction: advice,Sentence: The concomitant use of H2 blockers or proton pump inhibitors with SPRYCEL is not recommended.\nDrug1: proton pump inhibitors\nDrug2: SPRYCEL\nInteraction:,advice,"{advice, mechanism, none, effect, other}",Sentence: The use of antacids should be considered in place of H2 blockers or proton pump inhibitors in patients receiving SPRYCEL therapy.\nDrug 1: antacids\nDrug 2: H2 blockers\nInteraction: none\n\nSentence: The use of antacids should be considered in place of H2 blockers or proton pump inhibitors in patients receiving SPRYCEL therapy.\nDrug 1: antacids\nDrug 2: proton pump inhibitors\nInteraction: none\n\nSentence: The use of antacids should be considered in place of H2 blockers or proton pump inhibitors in patients receiving SPRYCEL therapy.\nDrug 1: antacids\nDrug 2: SPRYCEL\nInteraction: none\n\nSentence: The use of antacids should be considered in place of H2 blockers or proton pump inhibitors in patients receiving SPRYCEL therapy.\nDrug 1: H2 blockers\nDrug 2: proton pump inhibitors\nInteraction: none\n\nSentence: The use of antacids should be considered in place of H2 blockers or proton pump inhibitors in patients receiving SPRYCEL therapy.\nDrug 1: H2 blockers\nDrug 2: SPRYCEL\nInteraction: advice\n\nSentence: The concomitant use of H2 blockers or proton pump inhibitors with SPRYCEL is not recommended.\nDrug1: proton pump inhibitors\nDrug2: SPRYCEL\nInteraction:,none,"({'id': 'cmpl-4IU8yuPU9WAmvuVcrvDxgoAnQtZKU', 'object': 'text_completion', 'created': 1640311124, 'model': 'davinci:2020-05-03', 'choices': [{ ""finish_reason"": ""length"", ""index"": 0, ""logprobs"": { ""text_offset"": [ 1163 ], ""token_logprobs"": [ -0.23378392 ], ""tokens"": [ "" none"" ], ""top_logprobs"": [ { "" none"": -0.23378392 } ] }, ""text"": "" none"" }]}, Sentence: The use of antacids should be considered in place of H2 blockers or proton pump inhibitors in patients receiving SPRYCEL therapy.\nDrug 1: antacids\nDrug 2: H2 blockers\nInteraction: none\n\nSentence: The use of antacids should be considered in place of H2 blockers or proton pump inhibitors in patients receiving SPRYCEL therapy.\nDrug 1: antacids\nDrug 2: proton pump inhibitors\nInteraction: none\n\nSentence: The use of antacids should be considered in place of H2 blockers or proton pump inhibitors in patients receiving SPRYCEL therapy.\nDrug 1: antacids\nDrug 2: SPRYCEL\nInteraction: none\n\nSentence: The use of antacids should be considered in place of H2 blockers or proton pump inhibitors in patients receiving SPRYCEL therapy.\nDrug 1: H2 blockers\nDrug 2: proton pump inhibitors\nInteraction: none\n\nSentence: The use of antacids should be considered in place of H2 blockers or proton pump inhibitors in patients receiving SPRYCEL therapy.\nDrug 1: H2 blockers\nDrug 2: SPRYCEL\nInteraction: advice\n\nSentence: The concomitant use of H2 blockers or proton pump inhibitors with SPRYCEL is not recommended.\nDrug1: proton pump inhibitors\nDrug2: SPRYCEL\nInteraction:, effect other mechanism none advice )"
1157,17780,DDI-DrugBank.d31.s0.p0,MAO inhibitors should be used with caution in patients receiving hydralazine.,DDI-advise,DDI-DrugBank.d31,dev,MAO inhibitors,hydralazine,Sentence: MAO inhibitors should be used with caution in patients receiving hydralazine.\nDrug 1: MAO inhibitors\nDrug 2: hydralazine\nInteraction: advice,Sentence: MAO inhibitors should be used with caution in patients receiving hydrala

In [300]:
unique_label_dict.items(), unique_label_dict.values()

(dict_items([('Recommendation', 0), ('No Interaction', 1), ('Metabolic Mechanism ', 2), ('Effect Alteration', 3), ('Unspecified', 4)]),
 dict_values([0, 1, 2, 3, 4]))

In [301]:
df.groupby(filename_root).count()

,index,sentence,label,doc,split,entity1,entity2,prompts,empty_prompts,labels,unique_labels,test_ready_prompt,gpt3_output_ddi_test_prompts_random_33.5.50.perm_4.natural
ddi_test_prompts_random_33.5.50.perm_4.natural,,,,,,,,,,,,,
No Interaction,44,44,44,44,44,44,44,44,44,44,44,44,44
Recommendation,4,4,4,4,4,4,4,4,4,4,4,4,4


In [148]:
df.groupby('labels').count()

,index,sentence,label,doc,split,entity1,entity2,prompts,empty_prompts,unique_labels,test_ready_prompt,ddi_test_prompts_random_21.5.50.perm_4.natural,gpt3_output_ddi_test_prompts_random_21.5.50.perm_4.natural
labels,,,,,,,,,,,,,
Effect Alteration,12,12,12,12,12,12,12,12,12,12,12,12,12
Metabolic Mechanism,11,11,11,11,11,11,11,11,11,11,11,11,11
No Interaction,19,19,19,19,19,19,19,19,19,19,19,19,19
Recommendation,5,5,5,5,5,5,5,5,5,5,5,5,5
Unspecified,1,1,1,1,1,1,1,1,1,1,1,1,1


In [133]:
mea_df = df[['sentence',filename_root,'labels','test_ready_prompt']]

In [134]:
mea_df

,sentence,ddi_test_prompts_random_33.5.50.perm_4.natural,labels,test_ready_prompt
4386,The concomitant use of H2 blockers or proton pump inhibitors with SPRYCEL is not recommended.,Recommendation,Recommendation,"Sentence: Bentiromide may interact with acetaminophen (e.g., Tylenol), chloramphenicol (e.g., Chloromycetin), local anesthetics (e.g., benzocaine and lidocaine), para-aminobenzoic acid (PABA)-containing preparations (e.g., sunscreens and some multivitamins), procainamide (e.g., Pronestyl), sulfonamides (sulfa medicines), thiazide diuretics (use of these medicines during the test period will affect the test results), and pancreatic supplements (use of pancreatic supplements may give false test results).\nHow do drugs Bentiromide and Pronestyl interact based on the previous sentence?\nInteraction: Unspecified\n\nSentence: Combinations of clindamycin and gentamicin were indifferent for 16 and synergistic for 11 of the resistant strains. \nHow do drugs clindamycin and gentamicin interact based on the previous sentence?\nInteraction: Effect Alteration\n\nSentence: However, in a well-controlled study of patients with lymphoma on combination therapy, allopurinol did not increase the marrow toxicity of patients treated with cyclophosphamide, doxorubicin, bleomycin, procarbazine and/or mechlorethamine.\nHow do drugs procarbazine and mechlorethamine interact based on the previous sentence?\nInteraction: No Interaction\n\nSentence: Warfarin: Anticoagulant activity should be monitored, particularly in the first few days after initiating or changing VIOXX therapy in patients receiving warfarin or similar agents, since these patients are at an increased risk of bleeding complications.\nHow do drugs VIOXX and warfarin interact based on the previous sentence?\nInteraction: Recommendation\n\nSentence: Steady-state bosentan plasma concentrations were 3- to 4-fold higher than in the absence of cyclosporine A.\nHow do drugs bosentan and cyclosporine A interact based on the previous sentence?\nInteraction: Metabolic Mechanism \n\nSentence: The concomitant use of H2 blockers or proton pump inhibitors with SPRYCEL is not recommended.\nHow do drugs proton pump inhibitors and SPRYCEL interact based on the previous sentence?\nInteraction:"
17780,MAO inhibitors should be used with caution in patients receiving hydralazine.,No Interaction,Recommendation,"Sentence: Bentiromide may interact with acetaminophen (e.g., Tylenol), chloramphenicol (e.g., Chloromycetin), local anesthetics (e.g., benzocaine and lidocaine), para-aminobenzoic acid (PABA)-containing preparations (e.g., sunscreens and some multivitamins), procainamide (e.g., Pronestyl), sulfonamides (sulfa medicines), thiazide diuretics (use of these medicines during the test period will affect the test results), and pancreatic supplements (use of pancreatic supplements may give false test results).\nHow do drugs Bentiromide and Pronestyl interact based on the previous sentence?\nInteraction: Unspecified\n\nSentence: Combinations of clindamycin and gentamicin were indifferent for 16 and synergistic for 11 of the resistant strains. \nHow do drugs clindamycin and gentamicin interact based on the previous sentence?\nInteraction: Effect Alteration\n\nSentence: However, in a well-controlled study of patients with lymphoma on combination therapy, allopurinol did not increase the marrow toxicity of patients treated with cyclophosphamide, doxorubicin, bleomycin, procarbazine and/or mechlorethamine.\nHow do drugs procarbazine and mechlorethamine interact based on the previous sentence?\nInteraction: No Interaction\n\nSentence: Warfarin: Anticoagulant activity should be monitored, particularly in the first few days after initiating or changing VIOXX therapy in patients receiving warfarin or similar agents, since these patients are at an increased risk of bleeding complications.\nHow do drugs VIOXX and warfarin interact based on the previous sentence?\nInteraction: Recommendation\n\nSentence: Steady-stat

In [135]:
print(mea_df[mea_df['labels'] != mea_df[filename_root]].test_ready_prompt.values[0])

Sentence: Bentiromide may interact with acetaminophen (e.g., Tylenol), chloramphenicol (e.g., Chloromycetin), local anesthetics (e.g., benzocaine and lidocaine), para-aminobenzoic acid (PABA)-containing preparations (e.g., sunscreens and some multivitamins), procainamide (e.g., Pronestyl), sulfonamides (sulfa medicines), thiazide diuretics (use of these medicines during the test period will affect the test results), and pancreatic supplements (use of pancreatic supplements may give false test results).
How do drugs Bentiromide and Pronestyl interact based on the previous sentence?
Interaction: Unspecified

Sentence: Combinations of clindamycin and gentamicin were indifferent for 16 and synergistic for 11 of the resistant strains. 
How do drugs clindamycin and gentamicin interact based on the previous sentence?
Interaction: Effect Alteration

Sentence: However, in a well-controlled study of patients with lymphoma on combination therapy, allopurinol did not increase the marrow toxicity o

In [92]:
print(df.test_ready_prompt.values[0])

Sentence: Topical papaverine was used as a direct therapeutic action to manage vasospasm in a total of 11 patients .
Drugs: papaverine

Sentence: In summary , the chick embryo provides a reliable and simple experimental animal model of coniine - induced arthrogryposis .
Drugs: coniine

Sentence: Her QT interval returned to normal upon withdrawal of ketoconazole .
Drugs: ketoconazole

Sentence: Sodium bicarbonate alleviates penile pain induced by intracavernous injections for erectile dysfunction .
Drugs: sodium bicarbonate

Sentence: We report a patient in whom hypersensitivity to carbamazepine presented with generalized erythroderma , a severe leukemoid reaction , eosinophilia , hyponatremia , and renal failure .
Drugs: carbamazepine

Sentence: Plasma was tested for amphetamine and the cocaine metabolite benzoylecgonine using enzyme - mediated immunoassay methodology .
Drugs:


In [93]:
full_dev = pickle.load(open('../data/bc5cdr_chemical_test_prompts_full_dev_random_33.5., ','rb'))

In [94]:
conlleval_eval(full_dev.ner_seq,full_dev.bio_preds)

processed 116741 tokens with 5347 phrases; found: 5470 phrases; correct: 5114.
accuracy:  99.34%; (non-O)
accuracy:  99.34%; precision:  93.49%; recall:  95.64%; FB1:  94.55%
                X: precision:  93.49%; recall:  95.64%; FB1:  94.55%  5470
(93.49177330895795, 95.64241630821022, 94.55486733844874)


In [95]:
subset_with_predictions = full_dev.merge(df[['sents']],on='sents',how='inner')

In [96]:
conlleval_eval(subset_with_predictions.ner_seq,subset_with_predictions.bio_preds)

processed 5156 tokens with 218 phrases; found: 220 phrases; correct: 213.
accuracy:  99.59%; (non-O)
accuracy:  99.59%; precision:  96.82%; recall:  97.71%; FB1:  97.26%
                X: precision:  96.82%; recall:  97.71%; FB1:  97.26%  220
(96.81818181818181, 97.70642201834863, 97.26027397260275)


In [76]:
def run_gpt3(engine, prompt, logit_bias_text, sep):
    #constrain potential output to tokens in test sentence using logit bias
    logit_biases = {}
    tokens = tokenizer.encode(logit_bias_text)
    for token in tokens:
        logit_biases[token] = .1
    
    sep_token = tokenizer.encode(sep)
    logit_biases[sep_token[0]] = .1           
    
    #use API to generate completion
    sample = openai.Completion.create(engine=engine,
                                      prompt=prompt,
                                      max_tokens=10,
                                      temperature=0,
                                      logit_bias = logit_biases,
                                      presence_penalty=-0.001,
                                      stop=["\n", "<|endoftext|>"])
    return sample